In [1]:
import folium
import pandas as pd
import numpy as np
from pyproj import crs
import geopandas as gpd
import matplotlib.pyplot as plt

In [2]:
fp = 'data/shopping_centers.txt'
data = pd.read_csv(fp, sep=';') 

from geopandas.tools import geocode

# Geocode the addresses using Nominatim
geo = geocode(data['ADDRESS'], provider='nominatim', user_agent='autogis_CM', timeout=10)

data

,ID,NAME,ADDRESS
0,1,Itis,"Itäkatu 1-7, 00930 Helsinki"
1,2,Forum,"Mannerheimintie 20, 00100 Helsinki"
2,3,Iso-omena,"Piispansilta 11, 02230 Espoo"
3,4,Sello,"Leppävaarankatu 3-9, 02600 Espoo"
4,5,Jumbo,"Vantaanportinkatu 3, 01510 Vantaa"
5,6,REDI,"Hermannin rantatie 5, 00580 Helsinki"
6,7,Mall of Tripla,"Pasilansilta 11, 00520 Helsinki"


In [3]:
geo

,geometry,address
0,POINT (25.08294 60.21170),"Kauppakeskus Itis, 1-7, Itäkatu, Itäkeskus, Va..."
1,POINT (24.93858 60.16893),"Apollo Street Bar, 20, Mannerheimintie, Keskus..."
2,POINT (24.73779 60.16294),"Sports Academy, 11, Piispansilta, Matinkylän k..."
3,POINT (24.81042 60.21752),"Lasten kappeli Arkki, 3-9, Leppävaarankatu, Sä..."
4,POINT (24.96282 60.29245),"Stockmann, 3, Vantaanportinkatu, Pakkala, Avia..."
5,POINT (24.97904 60.18702),"Silta, 5, Hermannin rantatie, Verkkosaari, Kal..."
6,POINT (24.93440 60.19801),"Pasilansilta, Keski-Pasila, Pasila, Helsinki, ..."


In [4]:
# Convert points to GeoJSON
geo_gjson = folium.features.GeoJson(geo, name="Shopping centers")

In [5]:
# Create a Map instance
m = folium.Map(location=[60.20, 24.96], tiles = 'cartodbpositron', zoom_start=11, control_scale=True)

shopping_centers = ['Itis', 'Forum', 'IsoOmena', 'Sello', 'Jumbo', 'REDI', 'Tripla']
latitudes = [60.21170, 60.16893, 60.16294, 60.21752, 60.29245, 60.18702, 60.19801]
longitudes = [25.08294, 24.93858, 24.73779, 24.81042, 24.96282, 24.97904, 24.93440]

coords = list(zip(latitudes, longitudes))

from folium.plugins import MarkerCluster

marker_cluster = MarkerCluster(coords, name='Shopping centers', popups=shopping_centers, color='green').add_to(m)

#Show map
m

In [6]:
# Read text file into a variable called grid
grid = gpd.read_file('data/MetropAccess_YKR_grid_EurefFIN.shp')

# Check the data
grid

,x,y,YKR_ID,geometry
0,381875.0,6697880.0,5785640,"POLYGON ((382000.000 6697750.000, 381750.000 6..."
1,382125.0,6697880.0,5785641,"POLYGON ((382250.000 6697750.000, 382000.000 6..."
2,382375.0,6697880.0,5785642,"POLYGON ((382500.000 6697750.000, 382250.000 6..."
3,382625.0,6697880.0,5785643,"POLYGON ((382750.000 6697750.000, 382500.000 6..."
4,381125.0,6697630.0,5787544,"POLYGON ((381250.000 6697500.000, 381000.000 6..."
...,...,...,...,...
13226,372875.0,6665630.0,6016698,"POLYGON ((373000.000 6665500.000, 372750.000 6..."
13227,373125.0,6665630.0,6016699,"POLYGON ((373250.000 6665500.000, 373000.000 6..."
13228,372375.0,6665380.0,6018252,"POLYGON ((372500.000 6665250.000, 372250.000 6..."
13229,372625.0,6665380.0,6018253,"POLYGON ((372750.000 6665250.000, 372500.000 6..."


In [7]:
# Import module
import glob

# Fetch the filepaths to all textfiles found in data/ 
fp = r'dataM/'

# Create a list for all textfiles found in data/
filepaths = glob.glob(fp + "/*.txt")

In [8]:
# Create a for loop to iterate over the files in filepaths
for fp in filepaths:
    
    # Read the data
    data = pd.read_csv(fp, sep=';')
    
    # Select from_id and pt_r_t columns
    data = data[['from_id', 'pt_r_t']]
    
    # Rename from_id column to YKR_ID
    data = data.rename(columns={'from_id': 'YKR_ID'})
    
    # Create a variable to store shopping center names, obtained by splitting and and slicing the file names
    shopping_center_name = fp.split("_")[-1][:-4] 
    
    # Rename pt_r_t to include the shopping center names
    data = data.rename(columns={'pt_r_t': 'pt_r_t_' + shopping_center_name})
    
    # Merge the grid GeoDataFrame and the travel times using YKR_ID
    grid = grid.merge(data, on="YKR_ID")

In [9]:
# Remove no data values
data[data==-1]=np.nan
data.dropna(inplace=True)

In [10]:
# Create a new column for minimum travel time values 
grid['min_t'] = None

# Make a for loop to iterate over the rows
for i, row in grid.iterrows():
    
    # Create a variable to store all the columns needed
    value_columns = ['pt_r_t_Jumbo', 'pt_r_t_Dixi',
       'pt_r_t_Myyrmanni', 'pt_r_t_Itis', 'pt_r_t_Forum', 'pt_r_t_IsoOmena',
       'pt_r_t_Ruoholahti']
    
    # Find out the shortest travel time to any shopping center
    grid.loc[i, "min_t"] = row[value_columns].astype(float).min()
  
# Create a new column for the closest shopping center id values
grid['dominant_service'] = None

# Make a for loop to iterate over the rows
for i, row in grid.iterrows():
    
    # Create a variable to store all the columns needed
    value_columns = ['pt_r_t_Jumbo', 'pt_r_t_Dixi',
       'pt_r_t_Myyrmanni', 'pt_r_t_Itis', 'pt_r_t_Forum', 'pt_r_t_IsoOmena',
       'pt_r_t_Ruoholahti']
    # Find out the column name for the shortest travel time
    grid.loc[i, "dominant_service"] = row[value_columns].astype(float).idxmin()
    
grid

,x,y,YKR_ID,geometry,pt_r_t_Jumbo,pt_r_t_Dixi,pt_r_t_Myyrmanni,pt_r_t_Itis,pt_r_t_Forum,pt_r_t_IsoOmena,pt_r_t_Ruoholahti,min_t,dominant_service
0,381875.0,6697880.0,5785640,"POLYGON ((382000.000 6697750.000, 381750.000 6...",101,102,90,132,110,141,118,90,pt_r_t_Myyrmanni
1,382125.0,6697880.0,5785641,"POLYGON ((382250.000 6697750.000, 382000.000 6...",108,109,93,135,113,143,121,93,pt_r_t_Myyrmanni
2,382375.0,6697880.0,5785642,"POLYGON ((382500.000 6697750.000, 382250.000 6...",109,111,95,137,115,145,123,95,pt_r_t_Myyrmanni
3,382625.0,6697880.0,5785643,"POLYGON ((382750.000 6697750.000, 382500.000 6...",114,115,99,141,119,149,127,99,pt_r_t_Myyrmanni
4,381125.0,6697630.0,5787544,"POLYGON ((381250.000 6697500.000, 381000.000 6...",98,99,83,125,103,134,111,83,pt_r_t_Myyrmanni
...,...,...,...,...,...,...,...,...,...,...,...,...,...
13226,372875.0,6665630.0,6016698,"POLYGON ((373000.000 6665500.000, 372750.000 6...",109,79,95,79,52,38,48,38,pt_r_t_IsoOmena
13227,373125.0,6665630.0,6016699,"POLYGON ((373250.000 6665500.000, 373000.000 6...",110,81,97,81,54,40,50,40,pt_r_t_IsoOmena
13228,372375.0,6665380.0,6018252,"POLYGON ((372500.000 6665250.000, 372250.000 6...",-1,-1,-1,-1,-1,-1,-1,-1,pt_r_t_Jumbo
13229,372625.0,6665380.0,6018253,"POLYGON ((372750.000 6665250.000, 372500.000 6...",114,84,100,84,57,43,53,43,pt_r_t_IsoOmena


In [11]:
grid["geoid"] = grid.index.astype(str)

In [13]:
# Plot a choropleth map
# Notice: 'geoid' column that we created earlier needs to be assigned always as the first column
folium.Choropleth(
    geo_data=grid,
    name='Dominances',
    data=data,
    columns = ['geoid', 'dominant_service'],
    key_on='feature.id',
    fill_color='YlOrRd',
    fill_opacity=0.7,
    line_opacity=0.2,
    line_color='white',
    line_weight=0,
    highlight=False,
    smooth_factor=1.0,
    #threshold_scale=[100, 250, 500, 1000, 2000],
    legend_name= 'Minimum distance').add_to(m)

#Show map
m

KeyError: "None of ['geoid'] are in the columns"